# If you're going to change something in this notebook after running it, restart the kernel - otherwise, it'll do it for you

In [1]:
import os, sys, glob, pickle, tables
import numpy as np
# pip install simweights <--- run this once to install the package
# then replace <username> with your username and <version> with your Python version
sys.path.append('/home/tfutrell/.local/lib/python3.12/site-packages')
import simweights

In [2]:
'''All sim folders with h5 files:
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/O/22974_v2/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/O/22974_v3/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/O/22975_v2/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/O/22975_v3/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/O/22981_v2/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/O/22981_v3/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/O/22982_v2/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/O/22982_v3/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/O/22984_v2/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/O/22984_v3/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/O/22985_v2/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/O/22985_v3/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/O_allE_links/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/O_allE_links_v3/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/Si/22594_v2/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/Si/22594_v3/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/Si/22596_v2/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/Si/22596_v3/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/Si/22998_v3/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/Si/23000_v3/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/Si/23001_v3/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/Si/23002_v3/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/Si/23003_v3/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/Si_allE_links_v3/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p/22466_ITonly/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p/22467_ITonly/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p/22569_v2/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p/22569_v3/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p/22570_v2/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p/22570_v3/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p/22899_v2/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p/22899_v3/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p/22900_v2/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p/22900_v3/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p/22919_v2/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p/22919_v3/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p/22920_v2/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p/22920_v3/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p/22921_v2/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p/22921_v3/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p/22940_v2/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p/22940_v3/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p/22962_v2/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p/22962_v3/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p/22963_v2/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p/22963_v3/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p/22964_v2/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p/22964_v3/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p/22965_v2/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p/22965_v3/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p/23074_v3/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p/23080_v3/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p/23094_v3/h5files/*.h5
/data/ana/CosmicRay/IceTop_level3/sim/IC86.2018/SIBYLL2.3d/p_allE_links/h5files/*.h5
'''
p_list = glob.glob('/data/ana/CosmicRay/IceTop_level3/sim/IC86.2015/SIBYLL2.1/p/20174_v1s/h5files/*.h5')
he_list = glob.glob('/data/ana/CosmicRay/IceTop_level3/sim/IC86.2015/SIBYLL2.1/He/20178_v1s/h5files/*.h5')
o_list = glob.glob('/data/ana/CosmicRay/IceTop_level3/sim/IC86.2015/SIBYLL2.1/O/20179_v1s/h5files/*.h5')
fe_list = glob.glob('/data/ana/CosmicRay/IceTop_level3/sim/IC86.2015/SIBYLL2.1/Fe/20180_v1s/h5files/*.h5')

bigfilelist = p_list + he_list + o_list + fe_list

In [3]:
weighter = None
print('Getting weights...')
    
for filename in bigfilelist:
    file_obj = tables.open_file(filename, 'r')
    
    if weighter is None:
        weighter = simweights.IceTopWeighter(file_obj)
    else:
        weighter += simweights.IceTopWeighter(file_obj)

Getting weights...


In [4]:
PICKLING_FILES = {
    # Energy, type
    'energy': weighter.get_column('MCPrimary', 'energy'),
    'particle_type': weighter.get_column('MCPrimary', 'type'),
    # 'zenith': weighter.get_column('Laputop', 'zenith'),
    # Simulation weights
    'Hweights': weighter.get_weights(simweights.GaisserH4a_IT()),
    'Gweights': weighter.get_weights(simweights.GlobalSplineFit_IT()),
    # Cuts
    'hits': weighter.get_column('IceTopHLCSeedRTPulses_SnowUnAttenuated_info', 'nstrings'),
    'reco_pass': weighter.get_column('IT73AnalysisIceTopQualityCuts', 'IceTop_reco_succeeded')
}

In [5]:
for key, array in PICKLING_FILES.items():
    if not os.path.isfile(f'pickles/{key}.pkl'):
        print(f'Pickling {key}...')
        with open(f'pickles/{key}.pkl', 'wb') as f:
            pickle.dump(array, f)

print('Finished!')

Pickling energy...
Pickling particle_type...
Pickling Hweights...
Pickling Gweights...
Pickling hits...
Pickling reco_pass...
Finished!


: 